# Lawmaker Tracking
## Background
### Goal
There is a movement online of traders tracking the trades that lawmakers make in order to attempt to copy inside trading. My strategy is a little bit different. I am not sure whether or not lawmakers are inside trading, but I am certain that there is a movement of people that are making trades based on that information. My theory is there will be a jump in each stock as the time of discloser due to this wave of people buying. Potentially selling aswell. If I can find that pattern, there is something I can consistently trade on.

## Imports

In [1]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io
import re
import numpy as np
from pathlib import Path
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
marketDays = nyse.valid_days(start_date='2014-1-1', end_date='2021-12-22')
import warnings
warnings.filterwarnings('ignore')
import traceback
from sklearn.model_selection import train_test_split

## Methods

In [2]:
def importData(filename):
    df = pd.read_csv(filename)
    df.drop(df[df['amount'] == "Unknown"].index, inplace = True)
    df[['min_amount','max_amount']] = df.amount.str.split(" - ",expand=True,)
    df['min_amount'] = df["min_amount"].str.replace(r"[^0-9.]",'').astype(int)
    df.loc[df['min_amount'] == 1001, 'max_amount'] = "$15,000"
    df.loc[df['min_amount'] == 1000000, 'max_amount'] = "$5,000,000"
    df.loc[df['min_amount'] == 50000000, 'max_amount'] = "$1,000,000,000"
    df['max_amount'] = df["max_amount"].str.replace(r"[^0-9.]",'').astype(int)
    df = df.drop(['amount'], axis='columns')
    
    return df

In [3]:
def next_business_day(start_day):
    ONE_DAY = datetime.timedelta(days=1)
    temp_day = start_day
    next_day = temp_day
    while next_day.weekday() in [5,6] or next_day not in marketDays:
        next_day += ONE_DAY
        if next_day > datetime.date.today():
            return None
    temp_day = next_day
    return temp_day 

In [4]:
def addFeatures(dataset):
    featureData = pd.DataFrame()

    #Add Original Features
    featureData['Date'] = dataset['Date']
    featureData['Name'] = dataset['Name']
    featureData['Adj Close'] = dataset['Adj Close']
    featureData['Close'] = dataset['Close']
    featureData['High'] = dataset['High']
    featureData['Low'] = dataset['Low']
    featureData['Open'] = dataset['Open']
    featureData['Volume'] = dataset['Volume']
    
    
    #Add Day Before Features
    featureData['Adj Close'] = dataset['Adj Close'].shift(1)
    featureData['Close_DB'] = dataset['Close'].shift(1)
    featureData['Open_DB'] = dataset['Open'].shift(1)
    featureData['High_DB'] = dataset['High'].shift(1)
    featureData['Low_DB'] = dataset['Low'].shift(1)
    featureData['Volume_DB'] = dataset['Volume'].shift(1)

    #Add Close Average Windows
    featureData['Close_Average_Week'] = dataset['Close'].rolling(window=5).mean().shift(1)
    featureData['Close_Average_Month'] = dataset['Close'].rolling(window=21).mean().shift(1)
    featureData['Close_Average_3Month'] = dataset['Close'].rolling(window=63).mean().shift(1)
    featureData['Close_Average_Year'] = dataset['Close'].rolling(window=252).mean().shift(1)

    #Add Close Average Ratios
    featureData['Close_AVGRatio_Week_Month'] = featureData['Close_Average_Week']/featureData['Close_Average_Month']
    featureData['Close_AVGRatio_Week_3Month'] = featureData['Close_Average_Week']/featureData['Close_Average_3Month']
    featureData['Close_AVGRatio_Week_Year'] = featureData['Close_Average_Week']/featureData['Close_Average_Year']
    featureData['Close_AVGRatio_Month_3Month'] = featureData['Close_Average_Month']/featureData['Close_Average_3Month']
    featureData['Close_AVGRatio_Month_Year'] = featureData['Close_Average_Month']/featureData['Close_Average_Year']
    featureData['Close_AVGRatio_3Month_Year'] = featureData['Close_Average_3Month']/featureData['Close_Average_Year']

    #Add Volume Average Windows
    featureData['Volume_Average_Week'] = dataset['Volume'].rolling(window=5).mean().shift(1)
    featureData['Volume_Average_Month'] = dataset['Volume'].rolling(window=21).mean().shift(1)
    featureData['Volume_Average_3Month'] = dataset['Volume'].rolling(window=63).mean().shift(1)
    featureData['Volume_Average_Year'] = dataset['Volume'].rolling(window=252).mean().shift(1)

    #Add Volume Average Ratios
    featureData['Volume_AVGRatio_Week_Month'] = featureData['Volume_Average_Week']/featureData['Volume_Average_Month']
    featureData['Volume_AVGRatio_Week_3Month'] = featureData['Volume_Average_Week']/featureData['Volume_Average_3Month']
    featureData['Volume_AVGRatio_Week_Year'] = featureData['Volume_Average_Week']/featureData['Volume_Average_Year']
    featureData['Volume_AVGRatio_Month_3Month'] = featureData['Volume_Average_Month']/featureData['Volume_Average_3Month']
    featureData['Volume_AVGRatio_Month_Year'] = featureData['Volume_Average_Month']/featureData['Volume_Average_Year']
    featureData['Volume_AVGRatio_3Month_Year'] = featureData['Volume_Average_3Month']/featureData['Volume_Average_Year']
    
    #Add Close Standard Deviations
    featureData['Close_STD_Week'] = dataset['Close'].rolling(window=5).std().shift(1)
    featureData['Close_STD_Month'] = dataset['Close'].rolling(window=21).std().shift(1)
    featureData['Close_STD_3Month'] = dataset['Close'].rolling(window=63).std().shift(1)
    featureData['Close_STD_Year'] = dataset['Close'].rolling(window=252).std().shift(1)

    #Add Close Standard Deviations Ratios
    featureData['Close_STDRatio_Week_Month'] = featureData['Close_STD_Week']/featureData['Close_STD_Month']
    featureData['Close_STDRatio_Week_3Month'] = featureData['Close_STD_Week']/featureData['Close_STD_3Month']
    featureData['Close_STDRatio_Week_Year'] = featureData['Close_STD_Week']/featureData['Close_STD_Year']
    featureData['Close_STDRatio_Month_3Month'] = featureData['Close_STD_Month']/featureData['Close_STD_3Month']
    featureData['Close_STDRatio_Month_Year'] = featureData['Close_STD_Month']/featureData['Close_STD_Year']
    featureData['Close_STDRatio_3Month_Year'] = featureData['Close_STD_3Month']/featureData['Close_STD_Year']

    #Add Daily, Weekly, Monthly, Quarterly, and Yearly Returns
    featureData['Day_Return'] = ((dataset['Close']-dataset['Close'].shift(1))/dataset['Close'].shift(1)).shift(1)
    featureData['Week_Return'] = ((dataset['Close']-dataset['Close'].shift(5))/dataset['Close'].shift(5)).shift(1)
    featureData['Month_Return'] = ((dataset['Close']-dataset['Close'].shift(21))/dataset['Close'].shift(21)).shift(1)
    featureData['Quarter_Return'] = ((dataset['Close']-dataset['Close'].shift(63))/dataset['Close'].shift(63)).shift(1)
    featureData['Year_Return'] = ((dataset['Close']-dataset['Close'].shift(252))/dataset['Close'].shift(252)).shift(1)

    #Add Average of Daily Return
    featureData['Day_Return_Week_AVG'] = featureData['Day_Return'].rolling(window=5).mean()
    featureData['Day_Return_Month_AVG'] = featureData['Day_Return'].rolling(window=21).mean()
    featureData['Day_Return_3Month_AVG'] = featureData['Day_Return'].rolling(window=63).mean()
    featureData['Day_Return_Year_AVG'] = featureData['Day_Return'].rolling(window=252).mean()
    
    #Add Day After Features (Used for Exploratory Analysis)
    featureData['Close_DA'] = dataset['Close'].shift(-1)
    featureData['Open_DA'] = dataset['Open'].shift(-1)
    featureData['Volume_DA'] = dataset['Volume'].shift(-1)
    featureData['Close_WA'] = dataset['Close'].shift(-5)
    featureData['Open_WA'] = dataset['Open'].shift(-5)
    featureData['Volume_WA'] = dataset['Volume'].shift(-5)
    featureData['Close_MA'] = dataset['Close'].shift(-21)
    featureData['Open_MA'] = dataset['Open'].shift(-21)
    featureData['Volume_MA'] = dataset['Volume'].shift(-21)
    featureData['Change_Close_DA'] = (dataset['Close'].shift(-1)-dataset['Close'])/dataset['Close']
    featureData['Change_Close_WA'] = (dataset['Close'].shift(-5)-dataset['Close'])/dataset['Close']
    featureData['Change_Close_MA'] = (dataset['Close'].shift(-21)-dataset['Close'])/dataset['Close']

    featureData = featureData.dropna(axis=0)
    return featureData

## Pull In Data

In [5]:
houseData = importData('houseTrades.csv')
houseData = houseData.replace('--', np.NaN)
houseData = houseData.replace('nan', np.NaN)
houseData = houseData.dropna(subset=['owner', 'ticker'])
houseData.head()

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,representative,district,ptr_link,cap_gains_over_200_usd,min_amount,max_amount
0,2021,10/04/2021,2021-09-27,joint,BP,BP plc,purchase,Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,1001,15000
1,2021,10/04/2021,2021-09-13,joint,XOM,Exxon Mobil Corporation,purchase,Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,1001,15000
2,2021,10/04/2021,2021-09-10,joint,ILPT,Industrial Logistics Properties Trust - Common...,purchase,Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,15001,50000
3,2021,10/04/2021,2021-09-28,joint,PM,Phillip Morris International Inc,purchase,Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,15001,50000
4,2021,10/04/2021,2021-09-17,self,BLK,BlackRock Inc,sale_partial,Hon. Alan S. Lowenthal,CA47,https://disclosures-clerk.house.gov/public_dis...,False,1001,15000


In [6]:
senateData = importData('senateTrades.csv')
senateData = senateData.replace('--', np.NaN)
senateData = senateData.replace('nan', np.NaN)
senateData = senateData.dropna(subset=['owner', 'ticker'])
senateData.tail()

,transaction_date,owner,ticker,asset_description,asset_type,type,comment,senator,ptr_link,disclosure_date,min_amount,max_amount
8724,01/29/2014,Spouse,C,"Citigroup, Inc. (NYSE)",NaN,Sale (Partial),NaN,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,01/31/2014,1001,15000
8725,01/28/2014,Spouse,EBAY,eBay Inc. (NASDAQ),NaN,Sale (Partial),NaN,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,01/31/2014,1001,15000
8726,01/24/2014,Spouse,FB,"Facebook, Inc. (NASDAQ)",NaN,Purchase,NaN,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,01/31/2014,1001,15000
8727,01/24/2014,Spouse,CRM,"Salesforce.com, Inc (NYSE)",NaN,Purchase,NaN,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,01/31/2014,1001,15000
8728,01/24/2014,Spouse,GE,General Electric Company (NYSE),NaN,Sale (Partial),NaN,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,01/31/2014,1001,15000


## Combine Lawmaker Data
Not the most efficient way to do this, but it was easier for me to visualize this way and it doesn't take long enough to matter to me

In [7]:
lawmakerData = pd.DataFrame(columns=["date", "closest_trade_day", "ticker", "transaction", "disclosure", "self_owned", "joint_owned", "dependent_owned", "asset_description", "purchase", "sale_partial", "sale_full", "exchange", "lawmaker", "house", "senate", "link", "min_amount", "max_amount"])
for index, row in houseData.iterrows():
    try:
        td = pd.to_datetime(row['transaction_date'], format="%Y-%m-%d").date()
        tdnbd = next_business_day(pd.to_datetime(row['transaction_date'], format="%Y-%m-%d")).date()
    except:
        td = None
        tdnbd = None
    try:
        dd = pd.to_datetime(row['disclosure_date'], format="%m/%d/%Y").date()
        ddnbd = next_business_day(pd.to_datetime(row['disclosure_date'], format="%m/%d/%Y")).date()
    except:
        dd = None
        ddnbd = None
        
    try:
        lawmakerData = lawmakerData.append({'date': td,
                                           'closest_trade_day': tdnbd,
                                           'ticker': row['ticker'],
                                           'transaction': 1,
                                           'disclosure': 0,
                                           'self_owned': 1 if 'self' in row['owner'].lower() else 0,
                                           'joint_owned': 1 if 'joint' in row['owner'].lower() else 0,
                                           'dependent_owned': int(1) if ('spouse' in row['owner'].lower() or 'child' in row['owner'].lower() or 'dependent' in row['owner'].lower()) else int(0),
                                           'asset_description': row['asset_description'],
                                           'purchase': 1 if 'purchase' in row['type'].lower() else 0,
                                           'sale_partial': 1 if 'partial' in row['type'].lower() else 0,
                                           'sale_full': 1 if 'full' in row['type'].lower() else 0,
                                           'exchange': 1 if 'exchange' in row['type'].lower() else 0,
                                           'lawmaker': row['representative'],
                                           'house': 1,
                                           'senate': 0,
                                           'link': row['ptr_link'],
                                           'min_amount': row['min_amount'],
                                           'max_amount': row['max_amount']}, ignore_index=True)
        lawmakerData = lawmakerData.append({'date': dd,
                                           'closest_trade_day': ddnbd,
                                           'ticker': row['ticker'],
                                           'transaction': 0,
                                           'disclosure': 1,
                                           'self_owned': 1 if 'self' in row['owner'].lower() else 0,
                                           'joint_owned': 1 if 'joint' in row['owner'].lower() else 0,
                                           'dependent_owned': int(1) if ('spouse' in row['owner'].lower() or 'child' in row['owner'].lower() or 'dependent' in row['owner'].lower()) else int(0),
                                           'asset_description': row['asset_description'],
                                           'purchase': 1 if 'purchase' in row['type'].lower() else 0,
                                           'sale_partial': 1 if 'partial' in row['type'].lower() else 0,
                                           'sale_full': 1 if 'full' in row['type'].lower() else 0,
                                           'exchange': 1 if 'exchange' in row['type'].lower() else 0,
                                           'lawmaker': row['representative'],
                                           'house': 1,
                                           'senate': 0,
                                           'link': row['ptr_link'],
                                           'min_amount': row['min_amount'],
                                           'max_amount': row['max_amount']}, ignore_index=True)
    except Exception as e:
        print("House", row)
        print(e)
for index, row in senateData.iterrows():
    try:
        td = pd.to_datetime(row['transaction_date'], format="%m/%d/%Y").date()
        tdnbd = next_business_day(pd.to_datetime(row['transaction_date'], format="%m/%d/%Y")).date()
    except:
        td = None
        tdnbd = None
    try:
        dd = pd.to_datetime(row['disclosure_date'], format="%m/%d/%Y").date()
        ddnbd = next_business_day(pd.to_datetime(row['disclosure_date'], format="%m/%d/%Y")).date()
    except:
        dd = None
        ddnbd = None
    try:
        lawmakerData = lawmakerData.append({'date': td,
                                           'closest_trade_day': tdnbd,
                                           'ticker': row['ticker'],
                                           'transaction': 1,
                                           'disclosure': 0,
                                           'self_owned': 1 if 'self' in row['owner'].lower() else 0,
                                           'joint_owned': 1 if 'joint' in row['owner'].lower() else 0,
                                           'dependent_owned': int(1) if ('spouse' in row['owner'].lower() or 'child' in row['owner'].lower() or 'dependent' in row['owner'].lower()) else int(0),
                                           'asset_description': row['asset_description'],
                                           'purchase': 1 if 'purchase' in row['type'].lower() else 0,
                                           'sale_partial': 1 if 'partial' in row['type'].lower() else 0,
                                           'sale_full': 1 if 'full' in row['type'].lower() else 0,
                                           'exchange': 1 if 'exchange' in row['type'].lower() else 0,
                                           'lawmaker': row['senator'],
                                           'house': 0,
                                           'senate': 1,
                                           'link': row['ptr_link'],
                                           'min_amount': row['min_amount'],
                                           'max_amount': row['max_amount']}, ignore_index=True)
        lawmakerData = lawmakerData.append({'date': dd,
                                           'closest_trade_day': ddnbd,
                                           'ticker': row['ticker'],
                                           'transaction': 0,
                                           'disclosure': 1,
                                           'self_owned': 1 if 'self' in row['owner'].lower() else 0,
                                           'joint_owned': 1 if 'joint' in row['owner'].lower() else 0,
                                           'dependent_owned': int(1) if ('spouse' in row['owner'].lower() or 'child' in row['owner'].lower() or 'dependent' in row['owner'].lower()) else int(0),
                                           'asset_description': row['asset_description'],
                                           'purchase': 1 if 'purchase' in row['type'].lower() else 0,
                                           'sale_partial': 1 if 'partial' in row['type'].lower() else 0,
                                           'sale_full': 1 if 'full' in row['type'].lower() else 0,
                                           'exchange': 1 if 'exchange' in row['type'].lower() else 0,
                                           'lawmaker': row['senator'],
                                           'house': 0,
                                           'senate': 1,
                                           'link': row['ptr_link'],
                                           'min_amount': row['min_amount'],
                                           'max_amount': row['max_amount']}, ignore_index=True)
    except Exception as e:
        print("Senate", row)
        print(e)
    
lawmakerData.head()

,date,closest_trade_day,ticker,transaction,disclosure,self_owned,joint_owned,dependent_owned,asset_description,purchase,sale_partial,sale_full,exchange,lawmaker,house,senate,link,min_amount,max_amount
0,2021-09-27,2021-09-27,BP,1,0,0,1,0,BP plc,1,0,0,0,Hon. Virginia Foxx,1,0,https://disclosures-clerk.house.gov/public_dis...,1001,15000
1,2021-10-04,2021-10-04,BP,0,1,0,1,0,BP plc,1,0,0,0,Hon. Virginia Foxx,1,0,https://disclosures-clerk.house.gov/public_dis...,1001,15000
2,2021-09-13,2021-09-13,XOM,1,0,0,1,0,Exxon Mobil Corporation,1,0,0,0,Hon. Virginia Foxx,1,0,https://disclosures-clerk.house.gov/public_dis...,1001,15000
3,2021-10-04,2021-10-04,XOM,0,1,0,1,0,Exxon Mobil Corporation,1,0,0,0,Hon. Virginia Foxx,1,0,https://disclosures-clerk.house.gov/public_dis...,1001,15000
4,2021-09-10,2021-09-10,ILPT,1,0,0,1,0,Industrial Logistics Properties Trust - Common...,1,0,0,0,Hon. Virginia Foxx,1,0,https://disclosures-clerk.house.gov/public_dis...,15001,50000


In [8]:
lawmakerData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25704 entries, 0 to 25703
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               25701 non-null  object
 1   closest_trade_day  25701 non-null  object
 2   ticker             25704 non-null  object
 3   transaction        25704 non-null  object
 4   disclosure         25704 non-null  object
 5   self_owned         25704 non-null  object
 6   joint_owned        25704 non-null  object
 7   dependent_owned    25704 non-null  object
 8   asset_description  25700 non-null  object
 9   purchase           25704 non-null  object
 10  sale_partial       25704 non-null  object
 11  sale_full          25704 non-null  object
 12  exchange           25704 non-null  object
 13  lawmaker           25704 non-null  object
 14  house              25704 non-null  object
 15  senate             25704 non-null  object
 16  link               25704 non-null  objec

In [9]:
lawmakerData.isna().sum()

date                 3
closest_trade_day    3
ticker               0
transaction          0
disclosure           0
self_owned           0
joint_owned          0
dependent_owned      0
asset_description    4
purchase             0
sale_partial         0
sale_full            0
exchange             0
lawmaker             0
house                0
senate               0
link                 0
min_amount           0
max_amount           0
dtype: int64

## Some light data Preprocessing

In [10]:
tempData = lawmakerData.copy()
for lawmaker in tempData['lawmaker'].unique():
    cleanName = re.sub(r'[^A-Za-z ]+', '', lawmaker).replace(" ", "_")
    tempData['min_amount'] = np.where(tempData['sale_full']==1, -tempData["min_amount"], tempData["min_amount"])
    tempData['min_amount'] = np.where(tempData['sale_partial']==1, -tempData["min_amount"], tempData["min_amount"])
    tempData['max_amount'] = np.where(tempData['sale_full']==1, -tempData["max_amount"], tempData["max_amount"])
    tempData['max_amount'] = np.where(tempData['sale_partial']==1, -tempData["max_amount"], tempData["max_amount"])
    tempData[cleanName] = np.where(tempData['lawmaker']==lawmaker, (tempData["min_amount"] + tempData["max_amount"])/2, 0)
tempData = tempData.drop(['lawmaker', 'asset_description', 'link', 'min_amount', 'max_amount'], axis=1)
tempData['ID'] = tempData.apply(lambda row: str(row.closest_trade_day) + "-" + row.ticker, axis=1)
tempData.head()

,date,closest_trade_day,ticker,transaction,disclosure,self_owned,joint_owned,dependent_owned,purchase,sale_partial,...,Michael_F_Bennet,Patty_Murray,Joseph_Manchin_Iii,Chris_Van_Hollen,John_Cornyn,Maria_Cantwell,Michael__B_Enzi,Benjamin_L_Cardin,Cory_A_Booker,ID
0,2021-09-27,2021-09-27,BP,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2021-09-27-BP
1,2021-10-04,2021-10-04,BP,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2021-10-04-BP
2,2021-09-13,2021-09-13,XOM,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2021-09-13-XOM
3,2021-10-04,2021-10-04,XOM,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2021-10-04-XOM
4,2021-09-10,2021-09-10,ILPT,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2021-09-10-ILPT


In [11]:
trainableData = pd.DataFrame(columns=list(tempData))
compiledData = {'ID' : []}
for column in list(tempData):
    compiledData.update({column: []})

for ID in tempData['ID'].unique():
    IDUnique = tempData[tempData['ID'] == ID]
    compiledData['ID'].append(ID)
    compiledData['date'].append(IDUnique['date'].iloc[0])
    compiledData['ticker'].append(IDUnique['ticker'].iloc[0])
    compiledData['closest_trade_day'].append(IDUnique['closest_trade_day'].iloc[0])
    for key in list(compiledData.keys())[4:]:
        compiledData[key].append(IDUnique[key].sum())
            
trainableData = pd.DataFrame.from_dict(compiledData)
trainableData = trainableData.set_index('ID')
trainableData.describe()

,transaction,disclosure,self_owned,joint_owned,dependent_owned,purchase,sale_partial,sale_full,exchange,house,...,Tammy_Duckworth,Michael_F_Bennet,Patty_Murray,Joseph_Manchin_Iii,Chris_Van_Hollen,John_Cornyn,Maria_Cantwell,Michael__B_Enzi,Benjamin_L_Cardin,Cory_A_Booker
count,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,...,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.00000
mean,0.649616,0.649616,0.340073,0.674586,0.284573,0.674586,0.256874,0.357663,0.010109,0.620097,...,14.608522,-144.055853,130.214365,7.581935,10.614992,0.000000,4.043924,-0.808785,2.426355,-24.71760
std,0.724187,1.100177,0.741802,0.980397,0.570016,0.876980,0.725771,0.647072,0.110597,0.959069,...,444.035601,10690.280175,1012.361621,754.069071,495.798492,462.140428,331.648646,80.438525,139.309526,979.59606
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-750000.500000,0.000000,0.000000,0.000000,-32500.500000,0.000000,-8000.500000,0.000000,-75000.50000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,24.000000,27.000000,24.000000,27.000000,11.000000,15.000000,24.000000,17.000000,4.000000,28.000000,...,32500.500000,75000.500000,8000.500000,75000.500000,32500.500000,32500.500000,32002.000000,0.000000,8000.500000,0.00000


In [12]:
trainableData.head()

,date,closest_trade_day,ticker,transaction,disclosure,self_owned,joint_owned,dependent_owned,purchase,sale_partial,...,Tammy_Duckworth,Michael_F_Bennet,Patty_Murray,Joseph_Manchin_Iii,Chris_Van_Hollen,John_Cornyn,Maria_Cantwell,Michael__B_Enzi,Benjamin_L_Cardin,Cory_A_Booker
ID,,,,,,,,,,,,,,,,,,,,,
2021-09-27-BP,2021-09-27,2021-09-27,BP,1,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-04-BP,2021-10-04,2021-10-04,BP,0,1,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-13-XOM,2021-09-13,2021-09-13,XOM,1,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-04-XOM,2021-10-04,2021-10-04,XOM,0,1,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-10-ILPT,2021-09-10,2021-09-10,ILPT,1,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
trainableData.tail()

,date,closest_trade_day,ticker,transaction,disclosure,self_owned,joint_owned,dependent_owned,purchase,sale_partial,...,Tammy_Duckworth,Michael_F_Bennet,Patty_Murray,Joseph_Manchin_Iii,Chris_Van_Hollen,John_Cornyn,Maria_Cantwell,Michael__B_Enzi,Benjamin_L_Cardin,Cory_A_Booker
ID,,,,,,,,,,,,,,,,,,,,,
2014-01-24-FB,2014-01-24,2014-01-24,FB,1,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-31-FB,2014-01-31,2014-01-31,FB,0,1,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-24-CRM,2014-01-24,2014-01-24,CRM,1,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-31-CRM,2014-01-31,2014-01-31,CRM,0,1,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-24-GE,2014-01-24,2014-01-24,GE,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Collect Stock Data

In [14]:
# create empty dataframe
stock_combined = pd.DataFrame(columns=["Date", "Name", "Adj Close", "Close", "High", "Low", "Open", "Volume"])
listOfBadTickers = []
if not Path('stock_combined.csv').is_file():
    tickers = list(trainableData['ticker'].unique())
    start = datetime.datetime(2018,1,1)
    end = datetime.datetime(2021,12,20)
    # iterate over each symbol
    for i in tickers:  

        # print the symbol which is being downloaded
        print( str(tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  

        try:
            # download the stock price 
            stock = []
            stock = yf.download(i, threads = False, start=start, end=end, progress=True)
            stock.columns = ["Adj Close", "Close", "High", "Low", "Open", "Volume"]
            # append the individual stock prices 
            if len(stock) > 0:
                stock['Name'] = i
                stock["Date"] = pd.to_datetime(stock.index, format="%Y-%m-%d")
                stock = stock.reindex(columns=["Date", "Name", "Adj Close", "Close", "High", "Low", "Open", "Volume"])
                stock["index"] = stock["Date"]
                stock = stock.set_index('index')
                stock = stock.sort_index()
                stock_combined = stock_combined.append(addFeatures(stock), sort=False)
        except Exception as e:
            listOfBadTickers.append({'ticker':i, 'error': e})
            print(e)
            print(traceback.format_exc())
    stock_combined.to_csv("stock_combined.csv", index=False)
else:
    stock_combined = pd.read_csv("stock_combined.csv")

stock_combined['ID'] = stock_combined.apply(lambda row: str(pd.to_datetime(row.Date).date()) + "-" + row.Name, axis=1)
stock_combined.describe()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BOWX: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- VSM: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NPSND: None
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[***

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SPY160219P00180000: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ITC: None
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- QLIK: None
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CVC: None
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 compl

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DFRG: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CYBX: None
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CTRL: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CTCT: None
[*********************100%*****************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CLNY: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- KMP: None
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZMH: None
[*********************100%****************

,Adj Close,Close,High,Low,Open,Close_DB,Open_DB,High_DB,Low_DB,Volume_DB,...,Volume_DA,Close_WA,Open_WA,Volume_WA,Close_MA,Open_MA,Volume_MA,Change_Close_DA,Change_Close_WA,Change_Close_MA
count,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,...,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06
mean,9.431225e+01,9.559827e+01,9.317861e+01,9.442036e+01,9.258103e+01,9.550904e+01,9.248806e+01,9.309059e+01,9.433129e+01,2.566555e+07,...,2.571247e+07,9.602032e+01,9.301537e+01,2.582819e+07,9.723642e+01,9.426664e+01,2.619826e+07,1.063039e-03,4.951604e-03,1.932978e-02
std,2.041220e+02,2.069847e+02,2.017128e+02,2.044286e+02,2.037987e+02,2.067565e+02,2.035693e+02,2.014895e+02,2.042010e+02,6.640252e+08,...,6.648439e+08,2.081250e+02,2.049192e+02,6.671790e+08,2.115374e+02,2.082773e+02,6.726880e+08,6.994652e-02,9.488713e-02,1.827770e-01
min,1.000000e-04,1.000000e-04,1.000000e-04,1.000000e-04,-4.400000e-01,1.000000e-04,-4.400000e-01,1.000000e-04,1.000000e-04,0.000000e+00,...,0.000000e+00,1.000000e-04,-4.400000e-01,0.000000e+00,1.000000e-04,-4.400000e-01,0.000000e+00,-9.925000e-01,-9.945554e-01,-9.970501e-01
25%,2.304000e+01,2.340000e+01,2.269000e+01,2.305000e+01,2.203917e+01,2.340000e+01,2.202532e+01,2.268000e+01,2.304000e+01,2.023000e+05,...,2.022000e+05,2.344000e+01,2.208724e+01,2.020000e+05,2.352000e+01,2.221278e+01,2.029000e+05,-8.551312e-03,-1.971396e-02,-3.553307e-02
50%,4.901000e+01,4.964000e+01,4.842150e+01,4.904000e+01,4.757356e+01,4.961000e+01,4.753757e+01,4.839000e+01,4.901000e+01,7.729000e+05,...,7.723000e+05,4.978000e+01,4.772160e+01,7.716000e+05,5.012000e+01,4.812387e+01,7.745000e+05,3.658303e-04,3.309169e-03,1.496762e-02
75%,1.014000e+02,1.028400e+02,1.002700e+02,1.015401e+02,9.923000e+01,1.027300e+02,9.913938e+01,1.002100e+02,1.014500e+02,2.388400e+06,...,2.386900e+06,1.032900e+02,9.965514e+01,2.385602e+06,1.045000e+02,1.008174e+02,2.397300e+06,9.735024e-03,2.681988e-02,6.732486e-02
max,5.320000e+03,5.332080e+03,5.242090e+03,5.313570e+03,5.313570e+03,5.332080e+03,5.313570e+03,5.242090e+03,5.313570e+03,1.705493e+11,...,1.705493e+11,5.364040e+03,5.351400e+03,1.705493e+11,5.982450e+03,5.935720e+03,1.705493e+11,6.566666e+01,6.566666e+01,4.400000e+01


In [15]:
stock_combined = stock_combined.set_index('ID')
stock_combined.head()

,Date,Name,Adj Close,Close,High,Low,Open,Volume,Close_DB,Open_DB,...,Volume_DA,Close_WA,Open_WA,Volume_WA,Close_MA,Open_MA,Volume_MA,Change_Close_DA,Change_Close_WA,Change_Close_MA
ID,,,,,,,,,,,,,,,,,,,,,
2019-01-04-BP,2019-01-04,BP,38.959999,40.080002,39.419998,40.029999,33.186440,9336700,39.060001,32.175011,...,6114300.0,40.419998,33.360546,5746300.0,42.889999,35.499458,11124300.0,0.005739,0.008483,0.070110
2019-01-07-BP,2019-01-07,BP,39.570000,40.310001,39.580002,40.160000,33.294216,6114300,40.080002,33.186440,...,5654900.0,40.669998,33.509762,4976000.0,43.410000,35.681847,9903900.0,-0.001240,0.008931,0.076904
2019-01-08-BP,2019-01-08,BP,39.700001,40.259998,39.830002,40.029999,33.186440,5654900,40.310001,33.294216,...,6914500.0,40.650002,33.360546,4860400.0,43.259998,35.399979,8115800.0,0.008694,0.009687,0.074516
2019-01-09-BP,2019-01-09,BP,40.180000,40.610001,40.150002,40.360001,33.460022,6914500,40.259998,33.186440,...,4822500.0,40.220001,33.261051,4168900.0,42.500000,35.225880,4970500.0,0.003694,-0.009604,0.046540
2019-01-10-BP,2019-01-10,BP,40.389999,40.759998,40.130001,40.700001,33.741898,4822500,40.610001,33.460022,...,5746300.0,40.330002,33.269341,4345300.0,42.320000,34.993736,4730900.0,-0.008342,-0.010549,0.038273


## Combine Lawmaker and Stock Data

In [16]:
trainableData.head()

,date,closest_trade_day,ticker,transaction,disclosure,self_owned,joint_owned,dependent_owned,purchase,sale_partial,...,Tammy_Duckworth,Michael_F_Bennet,Patty_Murray,Joseph_Manchin_Iii,Chris_Van_Hollen,John_Cornyn,Maria_Cantwell,Michael__B_Enzi,Benjamin_L_Cardin,Cory_A_Booker
ID,,,,,,,,,,,,,,,,,,,,,
2021-09-27-BP,2021-09-27,2021-09-27,BP,1,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-04-BP,2021-10-04,2021-10-04,BP,0,1,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-13-XOM,2021-09-13,2021-09-13,XOM,1,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-04-XOM,2021-10-04,2021-10-04,XOM,0,1,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-10-ILPT,2021-09-10,2021-09-10,ILPT,1,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
combinedData = pd.concat([stock_combined, trainableData.rename(columns={"date": "action_date"}).drop(["ticker"], axis=1)], axis=1)

combinedData.describe()

,Adj Close,Close,High,Low,Open,Close_DB,Open_DB,High_DB,Low_DB,Volume_DB,...,Tammy_Duckworth,Michael_F_Bennet,Patty_Murray,Joseph_Manchin_Iii,Chris_Van_Hollen,John_Cornyn,Maria_Cantwell,Michael__B_Enzi,Benjamin_L_Cardin,Cory_A_Booker
count,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,1.091732e+06,...,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.000000,19784.00000
mean,9.431225e+01,9.559827e+01,9.317861e+01,9.442036e+01,9.258103e+01,9.550904e+01,9.248806e+01,9.309059e+01,9.433129e+01,2.566555e+07,...,14.608522,-144.055853,130.214365,7.581935,10.614992,0.000000,4.043924,-0.808785,2.426355,-24.71760
std,2.041220e+02,2.069847e+02,2.017128e+02,2.044286e+02,2.037987e+02,2.067565e+02,2.035693e+02,2.014895e+02,2.042010e+02,6.640252e+08,...,444.035601,10690.280175,1012.361621,754.069071,495.798492,462.140428,331.648646,80.438525,139.309526,979.59606
min,1.000000e-04,1.000000e-04,1.000000e-04,1.000000e-04,-4.400000e-01,1.000000e-04,-4.400000e-01,1.000000e-04,1.000000e-04,0.000000e+00,...,0.000000,-750000.500000,0.000000,0.000000,0.000000,-32500.500000,0.000000,-8000.500000,0.000000,-75000.50000
25%,2.304000e+01,2.340000e+01,2.269000e+01,2.305000e+01,2.203917e+01,2.340000e+01,2.202532e+01,2.268000e+01,2.304000e+01,2.023000e+05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,4.901000e+01,4.964000e+01,4.842150e+01,4.904000e+01,4.757356e+01,4.961000e+01,4.753757e+01,4.839000e+01,4.901000e+01,7.729000e+05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,1.014000e+02,1.028400e+02,1.002700e+02,1.015401e+02,9.923000e+01,1.027300e+02,9.913938e+01,1.002100e+02,1.014500e+02,2.388400e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,5.320000e+03,5.332080e+03,5.242090e+03,5.313570e+03,5.313570e+03,5.332080e+03,5.313570e+03,5.242090e+03,5.313570e+03,1.705493e+11,...,32500.500000,75000.500000,8000.500000,75000.500000,32500.500000,32500.500000,32002.000000,0.000000,8000.500000,0.00000


In [27]:
combinedData.head()

,Date,Name,Adj Close,Close,High,Low,Open,Volume,Close_DB,Open_DB,...,Tammy_Duckworth,Michael_F_Bennet,Patty_Murray,Joseph_Manchin_Iii,Chris_Van_Hollen,John_Cornyn,Maria_Cantwell,Michael__B_Enzi,Benjamin_L_Cardin,Cory_A_Booker
ID,,,,,,,,,,,,,,,,,,,,,
2019-01-04-BP,2019-01-04,BP,38.959999,40.080002,39.419998,40.029999,33.186440,9336700,39.060001,32.175011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07-BP,2019-01-07,BP,39.570000,40.310001,39.580002,40.160000,33.294216,6114300,40.080002,33.186440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08-BP,2019-01-08,BP,39.700001,40.259998,39.830002,40.029999,33.186440,5654900,40.310001,33.294216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-09-BP,2019-01-09,BP,40.180000,40.610001,40.150002,40.360001,33.460022,6914500,40.259998,33.186440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-10-BP,2019-01-10,BP,40.389999,40.759998,40.130001,40.700001,33.741898,4822500,40.610001,33.460022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
print(combinedData.isna().sum())

Date                    7919
Name                    7919
Adj Close               7919
Close                   7919
High                    7919
                      ...   
John_Cornyn          1079867
Maria_Cantwell       1079867
Michael__B_Enzi      1079867
Benjamin_L_Cardin    1079867
Cory_A_Booker        1079867
Length: 214, dtype: int64


### Handle N/As

In [29]:
combinedData = combinedData.dropna(subset=["Date", "Name", "Adj Close", "Close", "High", "Low", "Open", "Volume"])
for column in list(combinedData)[66:]:
    combinedData[column].fillna(value=0, inplace=True)
    if combinedData[column].sum() == 0:
        combinedData = combinedData.drop(column, axis=1)

## Save Data for Data Analysis

In [31]:
combinedData.to_csv("stocks_and_lawmakers.csv", index=True)